# Segmenting and Clustering Neighbourhoods in Toronto

## Introduction

In this notebook, we will complete the Week3 peer-graded assignment for the Applied Data Science Capstone course. The project requires us to segment and cluster neighbourhoods in Toronto using data available on this Wikipedia page.

[List of postal codes of Canada: M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

First, we will scrape the postal code data from the Wikipedia page using the BeautifulSoup package and clean it. Then, we will use the Geocoder package to add geographical coordinates to each neighbourhood. Next, we will use the Foursquare API to get data for each of these neighbourhoods. Finally, we will build a model that will use the details of each neighbourhood to create clusters of similar locations. 

## Table of Contents

1. Importing libraries and initial setup
2. Web scraping Toronto neighbourhood data
3. Data cleaning
4. Adding geographical coordinates
5. Visualizing neighbourhoods using Folium
6. Analyzing neighbourhoods using Foursquare API
7. Final analysis

## 1. Importing Libraries and Initial Setup

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import pgeocode
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from pandas import json_normalize
from sklearn.cluster import KMeans

# Foursquare API credentials (masked values)
CLIENT_ID = "AQTJCK1V**********"
CLIENT_SECRET = "2EMV3DNS**********"
VERSION = "20180604"

## 2. Web scraping Toronto neighbourhood data

To scrape data from the Wikipedia page, we will first write a function that takes an HTML table as input and returns a pandas dataframe.

In [2]:
def readDataframeFromHTML(htmlTable):
    htmlRows = htmlTable.find_all("tr")
    dataRows = []
    for tr in htmlRows:
        htmlCells = tr.find_all(re.compile(r"(th|td)"))
        drow = []
        for td in htmlCells:
            try:
                drow.append(td.text.replace("\n", ""))
            except:
                continue
        if len(drow) > 0:
            dataRows.append(drow)

    df = pd.DataFrame(dataRows[1:], columns=dataRows[0])
    return (df)


Now, we will fetch the Wikipedia page html using the **requests** library. Next we will use the **BeautifulSoup** library to parse the html and retrieve the hmtl table containing Postal Codes, Boroughs and Neighbourhoods for Canada. We will pass this html table to our function **readDataframeFromHTML** to get a pandas dataframe.

In [3]:
wikiURL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
htmlPage = requests.get(wikiURL)
soup = BeautifulSoup(htmlPage.text, "html.parser")
htmlTable = soup.find("table", attrs={"class":"wikitable"})
df = readDataframeFromHTML(htmlTable)

## 3. Data cleaning

Now that we have the data for the Canadian neighbourhoods, we will clean the data by removing rows where **Borough** is *Not assigned*. Also, rows where **Neighbourhood** is *Not assigned*, we will set the **Neighbourhood** same as the **Borough** for that entry. We will also ensure that there are no rows with the same **Postal Codes** as we will be using these to get the geographical coordinates at a later stage.

In [4]:
df = df[df["Borough"] != "Not assigned"].reset_index(drop=True)
df.loc[df["Neighbourhood"] == "Not assigned", "Neighbourhood"] =  df.loc[df["Neighbourhood"] == "Not assigned", "Borough"]

duplicateCodes = df.groupby(by="Postal Code").count().reset_index(drop=True)
print("Number of rows with duplicate Postal Codes = " + str(duplicateCodes[duplicateCodes["Borough"] > 1].shape[0]))

Number of rows with duplicate Postal Codes = 0


Finally, we will check the size of the dataframe.

In [5]:
print(df.shape)
df.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 4. Adding geographical coordinates

In order to analyze each neighbourhood, we first need their geographical coordinates. There are multiple libraries in Python that can be used for this purpose. We will be using the **pgeocode** library which allows us to set the country location and then pass the postal code to get the desired coordinates.

We will write a simple function that takes the postal code as input and returns a dictionary of Latitude, Longitude for it.

In [6]:
def getLatLongData(postalCode):
    geoObject = pgeocode.Nominatim("CA")
    location = geoObject.query_postal_code(postalCode)
    coordinates = {"Latitude": location.latitude, "Longitude": location.longitude}
    return (coordinates)

We will then call the function for each of the postal codes in our dataframe. Finally, we will add the latitude and longitude data to the original dataframe.

In [7]:
allCoords = df["Postal Code"].map(getLatLongData)
coordsDF = pd.DataFrame(allCoords.to_list())
df = pd.concat([df, coordsDF], axis = 1)
print(df.shape)
df.head()

(103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


We also need to check that we have updated all the coordinates correctly, i.e. there are no missing values in the data.

In [8]:
df[df["Latitude"].isna() | df["Longitude"].isna()]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
76,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


Since there is one Postal code for which coordinates we not available, we will update these values using the csv file provided as past of the assignment.

[http://cocl.us/Geospatial_data](https://cocl.us/Geospatial_data)

In [9]:
df.loc[df["Postal Code"] == "M7R", "Latitude"] = 43.6369
df.loc[df["Postal Code"] == "M7R", "Longitude"] = -79.6158

It is also mentioned in the assignment that we only need to analyze the Boroughs whose name contains *Toronto*. So we will filter our dataframe for these boroughs and recheck the shape.

In [10]:
torontoData = df[df["Borough"].str.contains("Toronto")].reset_index(drop=True)
print(torontoData.shape)
torontoData.head()

(40, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


## 5. Visualizing neighbourhoods using Folium

We will start by visualizing the map of Toronto and the neighbourhoods that we wish to analyze. We will use the **folium** package to do that. We need the coordinates of Toronto, which can be fetched by providing the address *Toronto, CA* to the **geolocator.geocode** function in the **geopy** package.

In [11]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="foursquare_api")
location = geolocator.geocode(address)
torontoLatitude = location.latitude
torontoLongitude = location.longitude
print('The geograpical coordinate of Toronta are {}, {}.'.format(torontoLatitude, torontoLongitude))

The geograpical coordinate of Toronta are 43.6534817, -79.3839347.


We can use these coordinates as the location of the **folium** map. We will add each neighbourhood in the **torontoData** dataframe as a circle on the map, using 1000 meters as the radius of each neighbourhood. We will also set the colors of each circle based on their *Borough*. This will help us visualize the locations of the neighbourhoods.

In [12]:
# Simple function that returns a dictionary of colors for each value in the variable list
def getRangeOfColors(varList):
    numColors = len(varList)
    x = np.arange(numColors)
    ys = [i + x + (i*x)**2 for i in range(numColors)]
    colorsArray = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = {}
    for i in range(numColors):
        rainbow[varList[i]] = colors.rgb2hex(colorsArray[i])
    return (rainbow)

In [13]:
# Function to create a folium map given location coordinates and marker data
def createFoliumMap(locLat, locLng, markerLats, markerLngs, markerColors, markerLabels, zoomStart, showScale, showCenter):
    rainbow = getRangeOfColors(markerColors.unique())
    fig = folium.Figure(width=800, height=500)
    # create map of Toronto using latitude and longitude values
    mapObject = folium.Map(location=[locLat, locLng], zoom_start=zoomStart, control_scale = showScale)
    # add markers to map
    for lat, lng, clr, lbl in zip(markerLats, markerLngs, markerColors, markerLabels):
        label = folium.Popup('{}, {}'.format(lbl, clr), parse_html=True)
        folium.Circle([lat, lng], radius=1000, color="none", fill=True, fill_color=rainbow[clr], fill_opacity=0.7, parse_html=False).add_to(mapObject)
        if showCenter:
            folium.CircleMarker([lat, lng], radius=2, popup=label, color="none", fill=True, fill_color=rainbow[clr], fill_opacity=1, parse_html=False).add_to(mapObject)

    fig.add_child(mapObject)
    return (fig)

In [14]:
createFoliumMap(
    locLat=torontoLatitude, 
    locLng=torontoLongitude, 
    markerLats=torontoData["Latitude"], 
    markerLngs=torontoData["Longitude"], 
    markerColors=torontoData["Borough"], 
    markerLabels=torontoData["Neighbourhood"], 
    zoomStart=11, showScale=True, showCenter=True
)

It looks like one of the neighbourhoods with postal code **M7Y** has been updated with incorrect geographical coordinates. We will manually update these values from the csv file provided as past of the assignment. 

In [15]:
torontoData[torontoData["Neighbourhood"] == "Business reply mail Processing Centre, South Central Letter Processing Plant Toronto"]


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
39,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505


In [16]:
torontoData.loc[torontoData["Postal Code"] == "M7Y", "Latitude"] = 43.6627439
torontoData.loc[torontoData["Postal Code"] == "M7Y", "Longitude"] = -79.321558


## 6. Analyzing neighbourhoods using Foursquare API

Here we will define some functions that can be used to retrieve the venues that are within a certain distance of some given geographical coordinates. We will then pass the coordiates of each of the neighbourhoods that we wish to analyze. The end result will be a list of venues corresponding to each neighbourhood as well as the category of the venue. 

**We will use this data to categorize our neighbourhoods - the underlying assumption is that two neighbourhoods can be said to be similar (or in the same cluster) if they have similar categories of venues in their vicinities.**

Since Toronto has a large area (630.2 sq.km.), and quite a few neighbourhoods have areas of more than 1-2 sq.kms., we will set **neighRadius** to 1000. This will result in substantial overlap between the neighbourhoods in denser areas like *Downtown Toronto*, but will better capture the characteristics of other neighbourhoods.

In [17]:
# function that extracts the category of the venue
def getVenueCategory(row):
    try:
        categoriesList = row['categories']
    except:
        categoriesList = row['venue.categories']
        
    if len(categoriesList) == 0:
        return None
    else:
        return (categoriesList[0]['name'])

In [18]:
def getVenuesForNeighbourhood(neighLatitude, neighLongitude, neighRadius = 1000, maxLimit = 100):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighLatitude, 
        neighLongitude, 
        neighRadius, 
        maxLimit)
    try:
        results = requests.get(url).json()
        # Convert the results json into a dataframe
        venues = results["response"]["groups"][0]["items"]
        nearbyVenues = json_normalize(venues) # flatten JSON
        filteredColumns = ["venue.name", "venue.categories", "venue.location.lat", "venue.location.lng"]
        nearbyVenues = nearbyVenues.loc[:, filteredColumns]
        # Set the column names by removing "venue." from the names
        nearbyVenues["venue.categories"] = nearbyVenues.apply(getVenueCategory, axis=1)
        nearbyVenues.columns = [col.split(".")[-1] for col in nearbyVenues.columns]
        return (nearbyVenues)
    except:
        return None
    

In [19]:
def getCompleteVenueList(names, latitudes, longitudes):
    venuesDF = pd.DataFrame()
    for name, lat, lng in zip(names, latitudes, longitudes):
        neighVenues = getVenuesForNeighbourhood(lat, lng)
        if neighVenues is None:
            print("No venues were found for {}.".format(name))
        else:
            neighVenues.insert(loc=0, column="neigh", value=name)
            venuesDF = venuesDF.append(pd.DataFrame(neighVenues))
    venuesDF.reset_index(drop=True)
    venuesDF.columns = ["Neighbourhood", "Venue", "Category", "Venue Latitude", "Venue Longitude"]
    return (venuesDF)



In [20]:
allVenues = getCompleteVenueList(names=torontoData["Neighbourhood"], latitudes=torontoData["Latitude"], longitudes=torontoData["Longitude"])
print("We found {} venues across {} neighbourhoods.".format(allVenues.shape[0], len(allVenues["Neighbourhood"].unique())))

We found 3286 venues across 40 neighbourhoods.


Below if the list of neighbourhoods along with the count of venues that were found in each of them.

In [21]:
allVenues.loc[:,["Neighbourhood", "Venue"]].groupby("Neighbourhood").count().sort_values(by="Venue", ascending=False).reset_index()

,Neighbourhood,Venue
0,Berczy Park,100
1,"High Park, The Junction South",100
2,"Toronto Dominion Centre, Design Exchange",100
3,"The Danforth West, Riverdale",100
4,"The Annex, North Midtown, Yorkville",100
5,Studio District,100
6,Stn A PO Boxes,100
7,St. James Town,100
8,"Richmond, Adelaide, King",100
9,"Regent Park, Harbourfront",100


We can also explore the data to check how many unique categories of venues are available.

In [22]:
print("There are {} uniques categories.".format(len(allVenues["Category"].unique())))

There are 285 uniques categories.


In order to cluster neighbourhoods based on venue categories, we will convert each category into a column. Then we will check how many venues of each category are found in each of the neighbourhoods. This will give us an idea of the overall composition of venues for each neighbourhood.

In [23]:
venuesOnehot = pd.get_dummies(allVenues[["Category"]], prefix="", prefix_sep="")
venuesOnehot.insert(loc=0, column="Neighbourhood", value=allVenues["Neighbourhood"])
print(venuesOnehot.shape)
venuesOnehot.head()

(3286, 286)


,Neighbourhood,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
venuesGrouped = venuesOnehot.groupby("Neighbourhood").mean().reset_index()
print(venuesGrouped.shape)
venuesGrouped.head()

(40, 286)


,Neighbourhood,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Berczy Park,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,...,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,...,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,"Business reply mail Processing Centre, South C...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020408,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,...,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.0
4,Central Bay Street,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,...,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0


We can also look at the top 10 most common venue categories for each neighbourhood.

In [25]:
def getMostCommonVenues(row, numTopVenues):
    rowCategories = row.iloc[1:]
    rowCategoriesSorted = rowCategories.sort_values(ascending=False)
    return rowCategoriesSorted.index.values[0:numTopVenues]

In [26]:
numTopVenues = 10
indicators = ["st", "nd", "rd"]
columns = ["Neighbourhood"]
for ind in np.arange(numTopVenues):
    try:
        columns.append("{}{} Most Common Venue".format(ind+1, indicators[ind]))
    except:
        columns.append("{}th Most Common Venue".format(ind+1))

neighVenuesSorted = pd.DataFrame(columns=columns)
neighVenuesSorted["Neighbourhood"] = venuesGrouped["Neighbourhood"]

for ind in np.arange(venuesGrouped.shape[0]):
    neighVenuesSorted.iloc[ind, 1:] = getMostCommonVenues(venuesGrouped.iloc[ind, :], numTopVenues)

neighVenuesSorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Japanese Restaurant,Hotel,Seafood Restaurant,Bakery,Park,Restaurant,Italian Restaurant,Gym
1,"Brockton, Parkdale Village, Exhibition Place",Restaurant,Bar,Coffee Shop,Café,Furniture / Home Store,Bakery,Park,Tibetan Restaurant,Gift Shop,Thrift / Vintage Store
2,"Business reply mail Processing Centre, South C...",Park,Pizza Place,Brewery,Italian Restaurant,Coffee Shop,Fast Food Restaurant,Sushi Restaurant,Bakery,Steakhouse,Diner
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Yoga Studio,Gym,Park,Dessert Shop,Furniture / Home Store,Sandwich Place,Italian Restaurant,Bakery
4,Central Bay Street,Coffee Shop,Café,Park,Hotel,Seafood Restaurant,Thai Restaurant,Bakery,Ramen Restaurant,Fast Food Restaurant,Furniture / Home Store
5,Christie,Korean Restaurant,Café,Grocery Store,Coffee Shop,Cocktail Bar,Ice Cream Shop,Mexican Restaurant,Caribbean Restaurant,Comedy Club,Restaurant
6,Church and Wellesley,Coffee Shop,Sushi Restaurant,Italian Restaurant,Park,Japanese Restaurant,Café,Diner,Grocery Store,Bookstore,Men's Store
7,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Theater,Restaurant,Japanese Restaurant,Seafood Restaurant,Bakery,Sushi Restaurant,Gastropub
8,Davisville,Italian Restaurant,Pizza Place,Sushi Restaurant,Indian Restaurant,Coffee Shop,Café,Sandwich Place,Bakery,Dessert Shop,Restaurant
9,Davisville North,Coffee Shop,Italian Restaurant,Café,Pizza Place,Sushi Restaurant,Restaurant,Gym,Park,Pharmacy,Supermarket


Now that we have analyzed the data, we can build a **k-means clustering** model to cluster similar neighbourhoods into clusters. We can experiment with different number of clusters to see which model gives better results. For now, we will set the number of clusters to 7. 

In [27]:
# set number of clusters
kclusters = 7
venuesGroupedClustering = venuesGrouped.drop("Neighbourhood", 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=4).fit(venuesGroupedClustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 3, 5, 0, 4, 5, 0, 3, 5])

In [28]:
# add clustering labels
neighVenuesSorted.insert(loc=0, column="Cluster Labels", value=kmeans.labels_)
mergedDF = torontoData
# merge with original neighbourhood data
mergedDF = mergedDF.join(neighVenuesSorted.set_index("Neighbourhood"), on="Neighbourhood", how="inner")
mergedDF.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,0,Coffee Shop,Park,Restaurant,Café,Theater,Bakery,Gastropub,Diner,Breakfast Spot,Pub
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,5,Coffee Shop,Park,Café,Sushi Restaurant,Italian Restaurant,Boutique,Hotel,Clothing Store,Japanese Restaurant,Pizza Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0,Coffee Shop,Gastropub,Japanese Restaurant,Café,Seafood Restaurant,Theater,Italian Restaurant,Hotel,Middle Eastern Restaurant,Bakery
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Gastropub,Italian Restaurant,Theater,Bakery,Art Gallery,Plaza
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,5,Pub,Coffee Shop,Pizza Place,Burger Joint,Grocery Store,Bar,Breakfast Spot,Health Food Store,Caribbean Restaurant,Japanese Restaurant


## 7. Final analysis

We have now assigned each neighbourhood to one of **7 clusters**. We can visualize the output of the model by using a **folium** map. This time, we will color each neighbourhood based on the cluster it has been assigned by the model.

In [29]:
createFoliumMap(
    locLat=torontoLatitude, 
    locLng=torontoLongitude, 
    markerLats=mergedDF["Latitude"], 
    markerLngs=mergedDF["Longitude"], 
    markerColors=mergedDF["Cluster Labels"], 
    markerLabels=mergedDF["Neighbourhood"], 
    zoomStart=11, showScale=True, showCenter=True
)

#### *By looking at the clusters, it can be seen that a lot of neighbourhoods in the central/ downtown part (PURPLE) of Toronto are marked in the same cluster. This is probably due to the high overlap in the venues between these densely located neighbourhoods. Neighbourhoods in the suburbs (LIGHT GREEN) on the East and West sides of Toronto are marked as similar. This completes the analysis of Toronto's neighbourhoods wherein we set out to segment and cluster neighbourhoods based on their venues.*